In [17]:

import re
import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None # we will overwrite some rows in the DataFrame
from pathlib import Path
# pykilosort 
from readSGLX import readMeta
from pykilosort import run, add_default_handler, np2_4shank_probe 


# get queue
root = r'\\zserver.cortexlab.net\Code\AVrig'
queue_csv_file = '%s\pykilosort_queue.csv' % root
queue_csv = pd.read_csv(queue_csv_file)
channel_map = np2_4shank_probe()
for idx,rec in queue_csv.iterrows():
    #check if recording is not being sorted already 
    if rec.sortedTag==0: 

        try:
            input_dir = Path(rec.ephysName)            
            # read the metadata and get the x- and ypos of channels 
            meta = readMeta(input_dir)
            channel_data = re.findall(r"\d+:\d+:\d+:\d+",meta['snsShankMap'])
            shank_idx = np.array([np.array(re.split(':',chandat)).astype('int')[0] for chandat in channel_data])
            side_idx = np.array([np.array(re.split(':',chandat)).astype('int')[1] for chandat in channel_data])
            channel_map.xc = ((shank_idx*250) + side_idx*32).astype('float64')
            channel_map.yc = (np.array([np.array(re.split(':',chandat)).astype('int')[2] for chandat in channel_data])*15).astype('float64')

            output_dir = input_dir.parents[0] / 'pyKS'
            output_dir.mkdir(parents=False, exist_ok=True)
            
            queue_csv.sortedTag.iloc[idx]= .5
            queue_csv.to_csv(queue_csv_file,index = False)
            add_default_handler(level='INFO') # print output as the algorithm runs
            run(input_dir, probe=channel_map, low_memory=True, dir_path=output_dir)
            queue_csv.sortedTag.iloc[idx]= 1
            queue_csv.to_csv(queue_csv_file,index = False)
        except: 
            queue_csv.sortedTag.iloc[idx]= -1
            queue_csv.to_csv(queue_csv_file,index = False)          



In [21]:
queue_csv

,ephysName,sortedTag
0,\\zinu.cortexlab.net\Subjects\AV009\2022-03-09...,5
1,\\zinu.cortexlab.net\Subjects\AV007\2022-03-12...,-1
2,\\zinu.cortexlab.net\Subjects\AV008\2022-03-12...,-1
3,\\zinu.cortexlab.net\Subjects\AV009\2022-03-15...,-1
4,\\zinu.cortexlab.net\Subjects\AV008\2022-03-18...,-1
...,...,...
76,\\zinu.cortexlab.net\Subjects\AV013\2022-06-10...,-2
77,\\zinu.cortexlab.net\Subjects\AV013\2022-06-10...,-2
78,\\zinu.cortexlab.net\Subjects\AV013\2022-06-10...,-2
79,\\zinu.cortexlab.net\Subjects\AV014\2022-06-10...,-2
